<a href="https://colab.research.google.com/github/log-ghj/webscraping/blob/main/Webscraping_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# some imports
import requests
from bs4 import BeautifulSoup
import urllib
from time import sleep
import pandas as pd

In [ ]:
# borrow the two functions from class
def get_soup(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup, r

In [ ]:
def find_next_url(soup, r, url):
  try:
    url_page = url[:-10]
    page = soup.find('li', attrs={'class': 'next'}).find('a')['href']
    next_url = urllib.parse.urljoin(url_page, page)
    return next_url
  except:
    return None

In [ ]:
# define a scraping function for a given category of books
def scraping(url):
  next_url = url
  url_page = next_url[:-10] 
  url_catalogue = 'https://books.toscrape.com/catalogue/'

  # containers
  all_titles = []
  all_prices = []
  all_categories = []
  all_available = []
  all_ratings = []
  all_descriptions = []
  all_pictures = []

  while next_url:
    print(next_url)
    soup, r = get_soup(next_url)
    
    # titles
    h3_list = soup.find_all('h3')
    titles = [title.get_text() for title in h3_list]
    all_titles.append(titles)

    # prices
    prices_list = soup.find_all('div', attrs={'class': 'product_price'})
    price_list = [price.find('p', attrs={'class': 'price_color'}) for price in prices_list]
    prices = [p.get_text().replace('Â', '') for p in price_list]
    all_prices.append(prices)

    # categories
    categories = soup.find('div', attrs={'class': 'page-header action'})
    categories = categories.get_text()
    categories = categories.replace('\n', '')
    categories = [categories]*len(h3_list)
    all_categories.append(categories)

    # information that needs to be gathered on each book's site
    new_url=[]
    links_list = soup.find_all('div', attrs={'class': 'image_container'})
    link_list = [link.find('a')['href'].replace('../../../', '') for link in links_list]

    for link in link_list:
      new_urls = url_catalogue+link
      new_url.append(new_urls)

    to_scrape = 'https://books.toscrape.com/'

    stars = []
    available_numbers = []
    descriptions = []
    pictures = []
    for url1 in new_url:
      soup1, r1 = get_soup(url1)

      # in stock
      available_n = soup1.find_all('p')[1].get_text().replace('\n','').replace('In stock (','').replace(' available)','').replace(' ','')
      available_numbers.append(available_n)
      
      # rating
      stars_n = soup1.find_all('p')[2]['class'][1]
      stars.append(stars_n)
      
      # description
      descr = soup1.find_all('p')[3].get_text()
      descriptions.append(descr)
      
      # picture
      picture = soup1.find('img')['src'].replace('../..','')
      picture_link = urllib.parse.urljoin(to_scrape,picture)
      pictures.append(picture_link)
    
    all_ratings.append(stars)
    all_available.append(available_numbers)
    all_descriptions.append(descriptions)
    all_pictures.append(pictures)

    next_url = find_next_url(soup, r, url)

  # flatten the lists
  all_available = [item for sublist in all_available for item in sublist]
  all_categories = [item for sublist in all_categories for item in sublist]
  all_descriptions = [item for sublist in all_descriptions for item in sublist]
  all_pictures = [item for sublist in all_pictures for item in sublist]
  all_prices = [item for sublist in all_prices for item in sublist]
  all_ratings = [item for sublist in all_ratings for item in sublist]
  all_titles = [item for sublist in all_titles for item in sublist]
  return all_titles, all_categories, all_descriptions, all_pictures, all_prices, all_ratings, all_available

In [ ]:
fiction = scraping('http://books.toscrape.com/catalogue/category/books/fiction_10/index.html')
names = ['Title','Category','Description','Picture','Price','Rating','In Stock']
fiction = pd.DataFrame(fiction).T
fiction.columns = names
fiction

http://books.toscrape.com/catalogue/category/books/fiction_10/index.html
http://books.toscrape.com/catalogue/category/books/fiction_10/page-2.html
http://books.toscrape.com/catalogue/category/books/fiction_10/page-3.html
http://books.toscrape.com/catalogue/category/books/fiction_10/page-4.html


,Title,Category,Description,Picture,Price,Rating,In Stock
0,Soumission,Fiction,"Dans une France assez proche de la nÃ´tre, un ...",https://books.toscrape.com/media/cache/ee/cf/e...,£50.10,One,20
1,Private Paris (Private #10),Fiction,Paris is burning--and only Private's Jack Morg...,https://books.toscrape.com/media/cache/05/87/0...,£47.61,Five,17
2,"We Love You, Charlie ...",Fiction,"The Freeman family--Charles, Laurel, and their...",https://books.toscrape.com/media/cache/87/a2/8...,£50.27,Five,16
3,Thirst,Fiction,"On a searing summer Friday, Eddie Chapman has ...",https://books.toscrape.com/media/cache/8c/54/8...,£17.27,Five,16
4,The Murder That Never ...,Fiction,"Given the opportunity, would you assume someon...",https://books.toscrape.com/media/cache/29/e0/2...,£54.11,Three,16
...,...,...,...,...,...,...,...
60,When I'm Gone,Fiction,"Dear Luke,First let me sayâI love youâ¦I di...",https://books.toscrape.com/media/cache/f8/dd/f...,£51.96,Three,1
61,The Silent Wife,Fiction,A chilling psychological thriller about a marr...,https://books.toscrape.com/media/cache/a6/37/a...,£12.34,Five,1
62,The Bette Davis Club,Fiction,"The morning of her nieceâs wedding, Margo Ju...",https://books.toscrape.com/media/cache/fb/6c/f...,£30.66,Three,1
63,Kitchens of the Great ...,Fiction,âA sweet and savory treat.â âPeopleâAn...,https://books.toscrape.com/media/cache/ed/be/e...,£57.20,Five,1


In [ ]:
historical_fiction = scraping('https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html')
names = ['Title','Category','Description','Picture','Price','Rating','In Stock']
historical_fiction = pd.DataFrame(historical_fiction).T
historical_fiction.columns = names
historical_fiction

https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html
https://books.toscrape.com/catalogue/category/books/historical-fiction_4/page-2.html


,Title,Category,Description,Picture,Price,Rating,In Stock
0,Tipping the Velvet,Historical Fiction,"""Erotic and absorbing...Written with starling ...",https://books.toscrape.com/media/cache/08/e9/0...,£53.74,One,20
1,Forever and Forever: The ...,Historical Fiction,A Proper Historical Romance seriesThe first bo...,https://books.toscrape.com/media/cache/f9/9e/f...,£29.69,Three,16
2,A Flight of Arrows ...,Historical Fiction,October 1776--August 1777It is said that what ...,https://books.toscrape.com/media/cache/ec/65/e...,£55.53,Five,16
3,The House by the ...,Historical Fiction,Anna is content with her well-ordered life in ...,https://books.toscrape.com/media/cache/46/a2/4...,£36.95,One,15
4,Mrs. Houdini,Historical Fiction,"A captivating debut novel, meticulously resear...",https://books.toscrape.com/media/cache/b4/45/b...,£30.25,Five,15
5,The Marriage of Opposites,Historical Fiction,From the New York Times bestselling author of ...,https://books.toscrape.com/media/cache/33/8e/3...,£28.08,Four,14
6,Glory over Everything: Beyond ...,Historical Fiction,From the author of the New York Times bestsell...,https://books.toscrape.com/media/cache/06/ac/0...,£45.84,Three,14
7,"Love, Lies and Spies",Historical Fiction,Juliana Telford is not your average nineteenth...,https://books.toscrape.com/media/cache/5c/19/5...,£20.55,Two,12
8,A Paris Apartment,Historical Fiction,Bienvenue Ã Paris!When April Vogt's boss tell...,https://books.toscrape.com/media/cache/b3/d9/b...,£39.01,Four,12
9,Lilac Girls,Historical Fiction,Inspired by the life of a real World War II he...,https://books.toscrape.com/media/cache/c0/80/c...,£17.28,Two,11


In [ ]:
politics = scraping('https://books.toscrape.com/catalogue/category/books/politics_48/index.html')
names = ['Title','Category','Description','Picture','Price','Rating','In Stock']
politics = pd.DataFrame(politics).T
politics.columns = names
politics

https://books.toscrape.com/catalogue/category/books/politics_48/index.html


,Title,Category,Description,Picture,Price,Rating,In Stock
0,Libertarianism for Beginners,Politics,Libertarianism isn't about winning elections; ...,https://books.toscrape.com/media/cache/91/a4/9...,£51.33,Two,19
1,Why the Right Went ...,Politics,âDionne's expertise is evident in this finel...,https://books.toscrape.com/media/cache/cd/64/c...,£52.65,Four,14
2,Equal Is Unfair: America's ...,Politics,Weâve all heard that the American Dream is v...,https://books.toscrape.com/media/cache/a6/3b/a...,£56.86,One,12


In [ ]:
fantasy = scraping('https://books.toscrape.com/catalogue/category/books/fantasy_19/index.html')
names = ['Title','Category','Description','Picture','Price','Rating','In Stock']
fantasy = pd.DataFrame(fantasy).T
fantasy.columns = names
fantasy

https://books.toscrape.com/catalogue/category/books/fantasy_19/index.html
https://books.toscrape.com/catalogue/category/books/fantasy_19/page-2.html
https://books.toscrape.com/catalogue/category/books/fantasy_19/page-3.html


,Title,Category,Description,Picture,Price,Rating,In Stock
0,Unicorn Tracks,Fantasy,After a savage attack drives her from her home...,https://books.toscrape.com/media/cache/9b/09/9...,£18.78,Three,16
1,"Saga, Volume 6 (Saga ...",Fantasy,"After a dramatic time jump, the three-time Eis...",https://books.toscrape.com/media/cache/4a/99/4...,£25.02,Three,16
2,Princess Between Worlds (Wide-Awake ...,Fantasy,Just as Annie and Liam are busy making plans t...,https://books.toscrape.com/media/cache/db/01/d...,£13.34,Five,16
3,Masks and Shadows,Fantasy,"The year is 1779, and Carlo Morelli, the most ...",https://books.toscrape.com/media/cache/05/c4/0...,£56.40,Two,16
4,Crown of Midnight (Throne ...,Fantasy,"""A line that should never be crossed is about ...",https://books.toscrape.com/media/cache/b7/7d/b...,£43.29,Three,16
5,Avatar: The Last Airbender: ...,Fantasy,Children are disappearing in the Fire Nation c...,https://books.toscrape.com/media/cache/9f/25/9...,£28.09,Two,16
6,A Court of Thorns ...,Fantasy,"A thrilling, seductive new series from New Yor...",https://books.toscrape.com/media/cache/0a/78/0...,£52.37,One,16
7,Throne of Glass (Throne ...,Fantasy,An alternate cover edition can be found here.A...,https://books.toscrape.com/media/cache/4e/8d/4...,£35.07,Three,15
8,The Glittering Court (The ...,Fantasy,"Big and sweeping, spanning from the refined pa...",https://books.toscrape.com/media/cache/2f/21/2...,£44.28,One,15
9,Hollow City (Miss Peregrineâs ...,Fantasy,Miss Peregrineâs Home for Peculiar Childrenw...,https://books.toscrape.com/media/cache/4c/fd/4...,£42.98,One,15


In [ ]:
philosophy = scraping('https://books.toscrape.com/catalogue/category/books/philosophy_7/index.html')
names = ['Title','Category','Description','Picture','Price','Rating','In Stock']
philosophy = pd.DataFrame(philosophy).T
philosophy.columns = names
philosophy

https://books.toscrape.com/catalogue/category/books/philosophy_7/index.html


,Title,Category,Description,Picture,Price,Rating,In Stock
0,Sophie's World,Philosophy,A page-turning novel that is also an explorati...,https://books.toscrape.com/media/cache/d7/39/d...,£15.94,Five,18
1,The Death of Humanity: ...,Philosophy,Do you believe human life is inherently valuab...,https://books.toscrape.com/media/cache/3f/f6/3...,£58.11,Four,16
2,The Stranger,Philosophy,This is an alternate cover edition for ISBN 06...,https://books.toscrape.com/media/cache/c8/1a/c...,£17.44,Four,15
3,Proofs of God: Classical ...,Philosophy,Leading theologian Matthew Levering presents a...,https://books.toscrape.com/media/cache/f6/cb/f...,£54.21,One,8
4,Kierkegaard: A Christian Missionary ...,Philosophy,Soren Kierkegaard (1813-1855) had a mission. T...,https://books.toscrape.com/media/cache/bf/90/b...,£47.13,One,8
5,At The Existentialist CafÃ©: ...,Philosophy,"Paris, near the turn of 1933. Three young frie...",https://books.toscrape.com/media/cache/ce/da/c...,£29.93,Five,7
6,Critique of Pure Reason,Philosophy,This entirely new translation of Critique of P...,https://books.toscrape.com/media/cache/3b/28/3...,£20.75,One,5
7,"Run, Spot, Run: The ...",Philosophy,A life shared with pets brings many emotions. ...,https://books.toscrape.com/media/cache/8c/bf/8...,£20.02,One,2
8,The Nicomachean Ethics,Philosophy,âOne swallow does not make a summer; neither...,https://books.toscrape.com/media/cache/16/f4/1...,£36.34,One,1
9,Meditations,Philosophy,"Written in Greek, without any intention of pub...",https://books.toscrape.com/media/cache/90/f7/9...,£25.89,Two,1
